In [10]:
from floridaman import data_cleaning
import pandas as pd
import numpy as np

In [11]:
# Load data
raw_data = data_cleaning.load('null_transformed')

In [12]:
# Generate sample dataset with the following parameters
# Source dataframe: raw_data
# Column drop threshold: 40%
# Column impute threshold: 20%
# n_neighbors for KNN imputation
this_data = data_cleaning.generate_candidate_dataset(raw_data, .40, .35, 5)

In [13]:
# Generate X and y dataframes for model input
X = np.array(this_data[data_cleaning.features(this_data)])
y = np.array(this_data['FAILURETYPE'])

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

max_score = 0

print("Training progress: ", end = '')
for COL_DROP in range(1,100,5):
    print('*', end = '')
    for IMPUTE in range(1, COL_DROP, 1):
        for NEIGHBOR in range(1, 20, 2):
            COL_DROP = COL_DROP/100
            IMPUTE = IMPUTE/100
            
            knn = KNeighborsClassifier(n_neighbors=NEIGHBOR)
            this_score = cross_val_score(knn, X, y, cv=8, scoring='roc_auc_ovo')
            this_score = this_score.mean()
            
            if (this_score > max_score):
                best_mode = [COL_DROP, IMPUTE, NEIGHBOR, this_score]

In [15]:
this_sample = data_cleaning.balance(this_data)

In [16]:
this_sample['FAILURETYPE'].value_counts()

2    161
1    161
0    161
Name: FAILURETYPE, dtype: int64

In [17]:
data_cleaning.fullNormData(this_data)

,roduid,UWI,lifetime_start,lifetime_end,FAILURETYPE,H2S_CONCENTRATION,PrimarySetpoint,SecondarySetpoint,StrokeLength,Fillage,...,rod_make,rod_apigrade,ROUTE,overall_max_sideload,shallow_max_sideload,max_unguided_sideload,DESANDDEGAS_TYP,NIPPLE_SET_DEPTH,pump_bore,lifetime
